## CATEGORICAL DATA

### BACKGROUND AND MOTIVATION

In [230]:
# unique/value_counts - extract the distinct values from an array and commute frequencies
import numpy as np
import pandas as pd

In [231]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [232]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [233]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [234]:
# dimension tables - contains distinct values
# stores the primary obs as integer keys referencing the dimension table

In [235]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])

In [236]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [237]:
dim

0     apple
1    orange
dtype: object

In [238]:
# use 'take' method to restore the original Series of strings
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [239]:
# this representation as integers is called the 'categorical' representation
# a.k.a dictionary-encoded
# array of distinct values can be called as categories/dictionary, or levels od the data
# integer values that refer the categories are called category codes

In [240]:
# categorical representation allows performance improvements for data analytics
# can perform transformations on categories without modifying codes

### CATEGORICAL TYPE IN PANDAS

In [241]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

In [242]:
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,13,3.030344
1,1,orange,3,3.262913
2,2,apple,3,2.576584
3,3,apple,8,1.390424
4,4,apple,12,3.781494
5,5,orange,11,1.175560
6,6,apple,11,0.977926
7,7,apple,12,1.372471


In [243]:
# df['fruit'] - array of Python string objs
# convert it to categorical
# values of fruit_cat are not NumPy array but instance of pandas.Categorical
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [244]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [245]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [246]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [247]:
# convert a DF column to categorical by assigning the converted result
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [248]:
# create pandas.Categorical from other types of Python sequences
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [249]:
# if categorical encoded data has been fetched from another source
# use from_codes constructor
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]

In [250]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [251]:
# without explicit params, categorical conversions assume no specific ordering of the categories
# when using from_codes, one can indicate category having a meaningful ordering
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [252]:
# convert unordered categorical instance to ordered
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [253]:
# categorical array can consist of any immutable value types

### COMPUTATIONS WITH CATEGORICALS

In [254]:
# groupby functions work better with categoricals
# some functions also utilize ordered flag

In [255]:
np.random.seed(12345)

In [256]:
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [257]:
# commute quartile binning of this data and extract some statistics
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [258]:
# quartile names using labels arg
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [259]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [260]:
bins.categories

Index(['Q1', 'Q2', 'Q3', 'Q4'], dtype='object')

In [261]:
# labeled bins categorical do not contain info about the bins edges in the data
# use groupby to extract summary stats
bins = pd.Series(bins, name="quartile")
results = pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index()

In [262]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [263]:
# quatile column maintains original categorical info
# including ordering from bins
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### BETTER PERFORMANCE WITH CATEGORICALS

In [264]:
# converting dataset to categorical can show substantial performance gains
# categorical version of DF column will use less memory

In [265]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N//4))

In [266]:
# now convert labels to categorical
categories = labels.astype('category')

In [267]:
labels.memory_usage()

80000080

In [268]:
categories.memory_usage()

10000272

In [269]:
# conversion to category is a one-time cost
%time _ = labels.astype('category')

CPU times: user 366 ms, sys: 22.9 ms, total: 389 ms
Wall time: 386 ms


In [270]:
# GroupBy ops can be faster with categoricals
# underlying algorithms use integer-based codes array instead of an array of strings

### CATEGORICAL METHODS

In [271]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [272]:
# 'cat' provides access to categorical methods
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [273]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [274]:
# if we know that the actual set of categories extend beyond four values
# use set_categories method to change them

In [275]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [276]:
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [277]:
# data remains unchanged
# new categories will be reflected in ops that use them
# value_counts respects the categories
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [278]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [279]:
# for large datasets, categoricals are used for memory savings and better performance
# after filtering large DF or Series, many categories may not appear
# use 'remove_unused_categories' method to trim unobserved categories

In [280]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [281]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

### CREATING DUMMY VARIABLES FOR MODELING

In [282]:
# often need of transforming categorical data to dummy variable arises
# one-hot encoding - creates a DF with column for each distinct category
# 0 or 1 as per occurance

In [283]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [284]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [285]:
# pandas.get_dummies - converts one-dimensional categorical data in a DF containing dummy variables

In [286]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## ADVANCED GROUPBY USE

### GROUP TRANSFORMS AND "UNWRAPPED" GROUPBYS

In [287]:
# apply - grouped ops for performing transformations
# transform - similar to apply but imposes more constraints on the kind of function to be used
# 1. produces a scalar value to be broadcast to the shape of the group
# 2. produce an obj of the same shape as the input group
# 3. must not mutate its input

In [288]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [289]:
# group means by key
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [290]:
# to produce a Series of same shape as df['value'] but with values replaced by the average grouped by 'key'
# pass the function lambda x: x.mean() to transform

In [291]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [292]:
# for built-in aggregation functions, pass string alias
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [293]:
g.transform('sum')

0     18.0
1     22.0
2     26.0
3     18.0
4     22.0
5     26.0
6     18.0
7     22.0
8     26.0
9     18.0
10    22.0
11    26.0
Name: value, dtype: float64

In [294]:
# transform also works with functions that return a Series
# results must be the same size as the input

In [295]:
# multiply each group by 2 using a lambda function
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [296]:
# compute ranks in descending order for each group
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [297]:
# group transformation function consisting of simple aggregations
def normalize(x):
    return (x - x.mean()) / x.std()

In [298]:
# use this with transform or apply to obtain equivalent results
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [299]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [300]:
# built-in aggregate functions like 'mean'/'sum' are faster than general apply function
# these are also 'fast past' when used with transform
# allows to perform unwrapped group ops
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [301]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [302]:
# unwrapped group ops may involve multiple group aggregations
# overall benefit of vectorized ops often outweighs this

### GROUPED TIME RESAMPLING

In [303]:
# resample method is semantically a group op based on a time intervalization

In [304]:
N = 15

In [305]:
times = pd.date_range('2017-05-20 00:00', periods=N, freq='1min')

In [306]:
df = pd.DataFrame({'time': times,
                   'value': np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [307]:
# index by 'time' and then resample
df.set_index('time').resample('5 min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [311]:
# if DF contains multiple time series, marked by additional group key column
df2 = pd.DataFrame({'time': times.repeat(3),
                    'key': np.tile(['a', 'b', 'c'], N),
                    'value': np.arange(N * 3.)})
df2[:9]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0
7,2017-05-20 00:02:00,b,7.0
8,2017-05-20 00:02:00,c,8.0


In [320]:
# to perform same resampling of each value of 'key', introduce pandas.TimeGrouper obj
time_key = pd.Grouper(freq='5min')

In [321]:
# set the time index. group by 'key' and time_key, and aggregate

In [322]:
resampled = df2.set_index('time').groupby(['key', time_key]).sum()
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [323]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


In [324]:
# constraint with TimeGrouper is that the time must be the index of the Series or DF

## TECHNIQUES FOR METHOD CHAINING

In [325]:
# applying sequence of transformations to a dataset results in temporary variables creation
# these are never used later in the analysis

In [327]:
# read pages 750-755 from the book to get this